In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import shap
import joblib

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.utils import shuffle

# ================================================================
# 1. Đọc và xử lý dữ liệu
# ================================================================
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Dataset.csv')
print("Dataset shape:", data.shape)

# Giả sử cột đầu tiên là nhãn, các cột còn lại là các đặc trưng đã trích xuất
target_col = data.columns[0]
X = data.iloc[:, 1:]
y = data[target_col]

# Nếu y là số liên tục và có nhiều giá trị, chuyển thành nhãn phân loại dựa trên median
if np.issubdtype(y.dtype, np.number) and y.nunique() > 10:
    threshold = y.median()
    y = np.where(y < threshold, 0, 1)

# Chia dữ liệu thành tập train và test (stratify để giữ tỉ lệ nhãn)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=42, stratify=y
)

# ================================================================
# 2. Adversarial Training: Tạo dữ liệu đối kháng từ tập train
# ================================================================
# Xác định top feature cần thêm nhiễu. Nếu dataset không có tên 'average_subdomain_length', 'url_length', 'entropy_of_url'
top_features = ['average_subdomain_length', 'url_length', 'entropy_of_url']
for f in top_features:
    if f not in X.columns:
        top_features = list(X.columns[:3])
        break
print("Top features for adversarial training:", top_features)

noise_factor = 0.05  # 5% noise
X_train_adv = X_train.copy()
for f in top_features:
    # Thêm nhiễu theo phân phối Gaussian
    X_train_adv[f] = X_train_adv[f] * (1 + np.random.normal(0, noise_factor, X_train_adv[f].shape))

# Kết hợp dữ liệu gốc và dữ liệu đối kháng
X_train_aug = pd.concat([X_train, X_train_adv], axis=0)
y_train_aug = np.concatenate([y_train, y_train])
X_train_aug, y_train_aug = shuffle(X_train_aug, y_train_aug, random_state=42)
print("Augmented training set shape:", X_train_aug.shape)

# Huấn luyện mô hình Random Forest trên tập dữ liệu mở rộng
rf_adv = RandomForestClassifier(
    n_estimators=1000,
    max_depth=100,
    min_samples_split=2,
    min_samples_leaf=1,
    max_features='log2',
    bootstrap=False,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1
)
rf_adv.fit(X_train_aug, y_train_aug)

# Đánh giá mô hình adversarial training trên tập test gốc
y_pred_adv = rf_adv.predict(X_test)
acc_adv = accuracy_score(y_test, y_pred_adv)
print("\n🔥 Adversarial Training Model Accuracy (Original Test): {:.2f}%".format(acc_adv * 100))
print("\n📌 Classification Report (Original Test - Adv Model):\n", classification_report(y_test, y_pred_adv))
print("\n📌 Confusion Matrix (Original Test - Adv Model):\n", confusion_matrix(y_test, y_pred_adv))

# ================================================================
# 3. Iterative Feature Re-selection: Đánh giá lại tầm quan trọng của feature bằng SHAP
# ================================================================
explainer_adv = shap.TreeExplainer(rf_adv)
# Tính SHAP values cho tập test
shap_values_adv = explainer_adv.shap_values(X_test)

# Tính trung bình giá trị SHAP tuyệt đối cho từng feature của lớp 1 (hoặc trung bình trên tất cả lớp)
shap_importance = np.abs(shap_values_adv[1]).mean(axis=0)
feature_importance_df = pd.DataFrame({
    'feature': X.columns,
    'shap_importance': shap_importance
}).sort_values(by='shap_importance', ascending=False)
print("\nFeature importance from adversarial model (SHAP):")
print(feature_importance_df.head(10))

# Chọn ra các feature có robust (ví dụ: các feature có SHAP importance trên ngưỡng 0.01)
robust_threshold = 0.01
robust_features = feature_importance_df[feature_importance_df['shap_importance'] > robust_threshold]['feature'].tolist()
print("Robust features after re-selection:", robust_features)

# Huấn luyện lại mô hình với tập robust features
X_train_reduced = X_train[robust_features]
X_test_reduced = X_test[robust_features]

rf_reduced = RandomForestClassifier(
    n_estimators=1000,
    max_depth=100,
    min_samples_split=2,
    min_samples_leaf=1,
    max_features='log2',
    bootstrap=False,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1
)
rf_reduced.fit(X_train_reduced, y_train)

y_pred_reduced = rf_reduced.predict(X_test_reduced)
acc_reduced = accuracy_score(y_test, y_pred_reduced)
print("\n🔥 Accuracy with robust features only: {:.2f}%".format(acc_reduced * 100))
print("\n📌 Classification Report (Robust Features):\n", classification_report(y_test, y_pred_reduced))
print("\n📌 Confusion Matrix (Robust Features):\n", confusion_matrix(y_test, y_pred_reduced))

# ================================================================
# 4. Dynamic Feature Weighting: Điều chỉnh trọng số các feature dựa trên SHAP
# ================================================================
# Lấy dictionary trọng số từ SHAP (dựa trên feature_importance_df)
feature_weights = feature_importance_df.set_index('feature')['shap_importance'].to_dict()

# Chuẩn hóa trọng số
total_weight = sum(feature_weights.values())
feature_weights_normalized = {f: w / total_weight for f, w in feature_weights.items()}
print("\nNormalized feature weights:")
print(feature_weights_normalized)

# Tạo tập dữ liệu weighted bằng cách nhân mỗi feature với trọng số tương ứng
X_train_weighted = X_train.copy()
X_test_weighted = X_test.copy()
for f in X.columns:
    if f in feature_weights_normalized:
        X_train_weighted[f] = X_train_weighted[f] * feature_weights_normalized[f]
        X_test_weighted[f] = X_test_weighted[f] * feature_weights_normalized[f]

# Huấn luyện lại mô hình trên dữ liệu weighted
rf_weighted = RandomForestClassifier(
    n_estimators=1000,
    max_depth=100,
    min_samples_split=2,
    min_samples_leaf=1,
    max_features='log2',
    bootstrap=False,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1
)
rf_weighted.fit(X_train_weighted, y_train)

y_pred_weighted = rf_weighted.predict(X_test_weighted)
acc_weighted = accuracy_score(y_test, y_pred_weighted)
print("\n🔥 Accuracy with dynamic feature weighting: {:.2f}%".format(acc_weighted * 100))
print("\n📌 Classification Report (Dynamic Weighting):\n", classification_report(y_test, y_pred_weighted))
print("\n📌 Confusion Matrix (Dynamic Weighting):\n", confusion_matrix(y_test, y_pred_weighted))

# ================================================================
# 5. Lưu lại mô hình (có thể lưu từng model riêng nếu cần)
# ================================================================
joblib.dump(rf_adv, 'rf_adv_model.pkl')
joblib.dump(rf_reduced, 'rf_reduced_model.pkl')
joblib.dump(rf_weighted, 'rf_weighted_model.pkl')
print("\nCác model đã được lưu thành công dưới tên: 'rf_adv_model.pkl', 'rf_reduced_model.pkl', và 'rf_weighted_model.pkl'.")


Dataset shape: (247950, 42)
Top features for adversarial training: ['average_subdomain_length', 'url_length', 'entropy_of_url']
Augmented training set shape: (446310, 41)

🔥 Adversarial Training Model Accuracy (Original Test): 97.17%

📌 Classification Report (Original Test - Adv Model):
               precision    recall  f1-score   support

           0       0.97      0.98      0.97     12854
           1       0.97      0.97      0.97     11941

    accuracy                           0.97     24795
   macro avg       0.97      0.97      0.97     24795
weighted avg       0.97      0.97      0.97     24795


📌 Confusion Matrix (Original Test - Adv Model):
 [[12556   298]
 [  403 11538]]
